In [5]:
import requests
import pandas as pd
import time

def fetch_opensea_assets(offset, category):
    url = "https://api.opensea.io/api/v1/assets"
    query = {
        "offset": offset,
        "limit": 50,
        "asset_category": category,
    }

    headers = {"X-API-KEY": "your_api_key_here"}

    response = requests.get(url, params=query, headers=headers)
    
    if response.status_code != 200:
        print(f"Error fetching assets: {response.status_code}")
        return None

    return response.json()

def fetch_opensea_events(token_id, asset_contract_address):
    url = "https://api.opensea.io/api/v1/events"
    query = {
        "asset_contract_address": asset_contract_address,
        "token_id": token_id,
        "event_type": "successful",
        "limit": 50,
    }
    response = requests.get(url, params=query)

    if response.status_code != 200:
        print(f"Error fetching events: {response.status_code}")
        return None

    return response.json()

def process_opensea_data(data):
    if not data:
        return pd.DataFrame()

    assets = []
    for asset in data['assets']:
        try:
            events = fetch_opensea_events(asset['token_id'], asset['asset_contract']['address'])
            latest_sale = events['asset_events'][0] if events and events['asset_events'] else None

            assets.append({
                # ... (same as before)
            })
        except KeyError:
            continue

    return pd.DataFrame(assets)

categories = ['gaming', 'music', 'art']

for category in categories:
    offset = 0
    dataframes = []

    while True:
        data = fetch_opensea_assets(offset, category)
        if not data or not data['assets']:
            break

        df = process_opensea_data(data)
        dataframes.append(df)

        offset += 50

        # Add a delay between requests to avoid hitting rate limits
        time.sleep(5)  # Adjust the delay (in seconds) as needed

    combined_df = pd.concat(dataframes)
    combined_df.to_csv(f"{category}_nfts.csv", index=False)

Error fetching assets: 403


ValueError: No objects to concatenate

In [9]:
import requests
import pandas as pd
import time

def fetch_rarible_items(offset, category):
    url = "https://api.rarible.com/protocol/v0.1/ethereum/nft/items"
    query = {
        "offset": offset,
        "limit": 50,
        "categories": category,
    }
    response = requests.get(url, params=query)

    if response.status_code != 200:
        print(f"Error fetching items: {response.status_code}")
        return None

    return response.json()

def fetch_rarible_orders(item_id):
    url = f"https://api.rarible.com/protocol/v0.1/ethereum/order/orders"
    query = {
        "itemId": item_id,
    }
    response = requests.get(url, params=query)

    if response.status_code != 200:
        print(f"Error fetching orders: {response.status_code}")
        return None

    return response.json()

def process_rarible_data(data):
    if not data:
        return pd.DataFrame()

    items = []
    for item in data['items']:
        try:
            item_id = item['id']
            orders = fetch_rarible_orders(item_id)

            sale_orders = [o for o in orders['orders'] if o['type'] == 'SELL']
            bid_orders = [o for o in orders['orders'] if o['type'] == 'BID']

            latest_sale_order = sale_orders[0] if sale_orders else None
            latest_bid_order = bid_orders[0] if bid_orders else None

            items.append({
                "NFT ID": item_id,
                "Name": item['meta']['name'],
                "Description": item['meta']['description'],
                "Image": item['meta']['image']['url'],
                "Collection": item['collection']['name'],
                "Category": ','.join(item['categories']),
                "Creator": item['mintedBy'],
                "Owner": item['owners'][0],
                "Sale Date": latest_sale_order['createdAt'] if latest_sale_order else None,
                "Sale Price": latest_sale_order['make']['value'] if latest_sale_order else None,
                "Sale Currency": latest_sale_order['make']['assetType']['symbol'] if latest_sale_order else None,
                "Buyer": latest_sale_order['taker'] if latest_sale_order else None,
                "Seller": latest_sale_order['maker'] if latest_sale_order else None,
                "Bid Date": latest_bid_order['createdAt'] if latest_bid_order else None,
                "Bid Price": latest_bid_order['take']['value'] if latest_bid_order else None,
                "Bid Currency": latest_bid_order['take']['assetType']['symbol'] if latest_bid_order else None,
                "Bidder": latest_bid_order['maker'] if latest_bid_order else None,
            })
        except KeyError:
            continue

    return pd.DataFrame(items)


categories = ['gaming', 'music', 'art']

for category in categories:
    offset = 0
    dataframes = []

    while True:
        data = fetch_rarible_items(offset, category)
        if not data or not data['items']:
            break

        df = process_rarible_data(data)
        dataframes.append(df)

        offset += 50

        # Add a delay between requests to avoid hitting rate limits
        time.sleep(1)  # Adjust the delay (in seconds) as needed

    combined_df = pd.concat(dataframes)
    combined_df.to_csv(f"{category}_nfts_rarible_price_discovery.csv", index=False)


ConnectionError: HTTPSConnectionPool(host='api.rarible.com', port=443): Max retries exceeded with url: /protocol/v0.1/ethereum/nft/items?offset=0&limit=50&categories=gaming (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000196E23DBC10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))